<< Build helper functions to retrieve user's transactions info from Plaid data >>

In [ ]:
# High-level overview of Mayllon's API interactions between Matteo and me.

# @app.route('/account/transactions', methods=['POST'])
# def model_A():

#     token = requests.get('plaid_token', None)


#     txn = plaid_transactions()
#     bal = plaid_balance()
#     inv = plaid_inv()

#     if ():
#         output = model_A(txn, bal, inv)

#         return make_response(output)

In [ ]:
# Import libraries
import json
import requests
import pandas as pd
from datetime import datetime
from datetime import timedelta
from dotenv import load_dotenv


### Plaid PRODUCTS:
Transactions, Auth, Balance, Identity, Investment

In [ ]:
# Import environment variables

load_dotenv()
PLAID_CLIENT_ID = os.getenv('CLIENT_ID')
PLAID_SECRET = os.getenv('SECRET_KEY')
access_token= os.getenv('ACCESS_TOKEN')

In [ ]:
# Fetch Sandbox data from Plaid using 'request' library

url1 = "https://sandbox.plaid.com/transactions/get"
url2 = "https://sandbox.plaid.com/auth/get"
url3 = "https://sandbox.plaid.com/accounts/balance/get"
url4 = "https://sandbox.plaid.com/identity/get"
url5 = "https://sandbox.plaid.com/investments/holdings/get"


payload1 = {
          "client_id": PLAID_CLIENT_ID,
          "secret": PLAID_SECRET, 
          "access_token": access_token,
          "start_date": (datetime.now() - timedelta(days=30)).date().strftime("%Y-%m-%d"),
          "end_date": datetime.now().date().strftime("%Y-%m-%d")
           }

payload2 = {
          "client_id": PLAID_CLIENT_ID,
          "secret": PLAID_SECRET, 
          "access_token": access_token,
           }

headers = { "Content-Type": "application/json"}


txn = requests.post(url=url1, data=json.dumps(payload1),headers=headers).json()
auth = requests.post(url=url2, data=json.dumps(payload2),headers=headers).json()
balance = requests.post(url=url3, data=json.dumps(payload2),headers=headers).json()
id = requests.post(url=url4, data=json.dumps(payload2),headers=headers).json()
investment = requests.post(url=url5, data=json.dumps(payload2),headers=headers).json()


In [ ]:
a = 'accounts'
b = 'item'

print(auth[a]==txn[a], balance[b]==id[b]==investment[b])

### Metric #1: Transactions count and volume

In [ ]:
### Metric #1: Transactions count and volume
def txn(data):
    """
    Calculate tot txn count & tot txn volume
    
    Args:
        transaction data as a dict
        
    Returns:
        a tuple containing the cumulative number of transactions and the tot of inflow-outflow of transactions
    """
    try:
        cnt = data['total_transactions']
        date = []
        volume = []
        for i in range(len(data['transactions'])):
            txn = data['transactions'][i]
            v = txn['amount']
            volume.append(v)
        return (cnt, sum(volume))
        
    except Exception as e:
        pass



def net_monthly(data):
    """
    What is the net monthly inflow across all accounts for a given user?
    This function outputs the monthly net flow of money in/out the user's bank accounts
     
    Args:
        transaction data as a dict
    
    Returns:
        list of list of dates and net monthly flow
    """
    try:
        date = []
        amount = []
        for i in range(len(data['transactions'])):
            txn = data['transactions'][i]
            date.append(datetime.strptime(txn['date'], '%Y-%m-%d').date())
            amount.append(txn['amount'])

        df0 = pd.DataFrame(data={'amount': amount}, index=pd.DatetimeIndex(pd.to_datetime(date)))
        df1 = df0.groupby(pd.Grouper(freq='M', how='start')).sum()

        net_pair = []
        for i in range(len(df1)):
            dt = df1.index.to_list()[i].isoformat().split("T")[0]
            am = df1['amount'].to_list()[i]
            net_pair.append([dt,am])
        return net_pair
    except Exception as e:
        pass

### Metric #2: Length of credit history

### Metric #3: Avg age of borrower's credit accounts

### Metric #4: Composition or mix of borrower's accounts

In [ ]:
def acc_mix(data):
    """
    returns composition and status of user's accounts
    
    Agrs:
        data as dict
    
    Returns:
        number of accounts, account types, balances, and currencies as a list of lists
    """
    try: 
        types = []
        balances = []
        currency = []

        account = data["accounts"]
        for i in range(len(account)):
            t = str(account[i]['type']) + "_" + str(account[i]['subtype'])
            b = account[i]['balances']['current']
            c = account[i]['balances']['iso_currency_code'] 
            types.append(t)
            balances.append(b)
            currency.append(c)
            acc_mix = [len(types), types, balances, currency]
        return acc_mix

    except Exception as e:
        pass
    


### Metric #5: Credit Utilization Ratio

In [ ]:
def keep_credit(data):
    """
    Returns index of the user's credit accounts
    
    Args:
        transaction data as dict
    
    Return:
        list of indeces of all credit accounts 
    """
    credit_account = []
    try: 
        types = []

        account = data["accounts"]
        for i in range(len(account)):
            t = str(account[i]['type']) + "_" + str(account[i]['subtype'])
            if t in ['credit_credit card', 'depository_cd']:
                types.append(t)
                credit_account.append(i)
        return credit_account
    except Exception as e:
        pass


def credit_util_ratio(data):
    """
    Returns the credit utilization ratio = credit used/credit available
    
    Args:
    transaction datat as dict
    
    Returns:
    ratio as a float
    """
    try:
        tot_credit = []
        used_credit = []
        types = []

        account = data["accounts"]
        c = keep_credit(data)

        for i in c:
            tot = account[i]['balances']['current']
            t = str(account[i]['type']) + "_" + str(account[i]['subtype'])
            if account[i]['balances']['available'] != None:
                avail = account[i]['balances']['available']
            else: 
                avail = 0
            used = tot-avail
            tot_credit.append(tot)
            used_credit.append(used)
            types.append(t)
            r = sum(used_credit)/sum(tot_credit)
        return r
        
    except Exception as e:
        pass


### Run functions 
to extrapolate metrics

In [ ]:
txn(txn)
net_monthly(txn)
acc_mix(txn)
credit_util_ratio(txn)